In [16]:
import json
import numpy as np
import pandas as pd
import glob
import ast
import re
from tqdm import tqdm
import os.path
import collections
from collections import defaultdict


In [53]:
# def extract_list_from_string(input_string):
#     if 'NA' in input_string:
#         return 'NA'

#     # Clean up the input string by removing double parentheses
#     cleaned_string = re.sub(r'\(\(', r'(', input_string)
#     cleaned_string = re.sub(r'\)\)', r')', cleaned_string)

#     # Find the last occurrence of '[' and ']' to extract the last list
#     start_index = cleaned_string.rfind('[')
#     end_index = cleaned_string.rfind(']')

#     # Extract the substring between the '[' and ']'
#     list_string = cleaned_string[start_index:end_index + 1]

#     # Remove any newline characters and leading/trailing whitespace
#     list_string = list_string.replace('\n', '').strip()

#     # Convert the string to a list of tuples
#     list_of_tuples = eval(list_string)

#     # Return the list of tuples
#     return list_of_tuples

def regex_cleaner(input_string):
    pattern = r'\(\((.*?)\)\)'
    output_string = re.sub(pattern, r'(\g<1>)', input_string)
    return output_string

def extract_list_from_string(input_string):
    if 'NA' in input_string:
        return 'NA'

    # Find the last occurrence of '[' and ']' to extract the last list
    start_index = input_string.rfind('[')
    end_index = input_string.rfind(']')

    # Extract the substring between the '[' and ']'
    list_string = input_string[start_index + 1:end_index]

    # Remove any newline characters and leading/trailing whitespace
    list_string = list_string.replace('\n', '').strip()

    # Use a regular expression to match keys and values in the input string
    # tuple_pattern = re.compile(r"\s*'([^']*)'\s*,\s*(-?\d+)\s*")
    tuple_pattern = re.compile(r"\(([^,]*),\s*(-?\d+)\)")
    matches = tuple_pattern.findall(list_string)

    # Convert the matches into a list of tuples
    list_of_tuples = [(key, int(value)) for key, value in matches]

    # Return the list of tuples
    return list_of_tuples



def extract_values_from_list(input_list):
    # Check if the list contains 'NA'
    if 'NA' in input_list:
        return 'NA'
    
    # Extract the values from the list of tuples
    values_list = []
    for tup in input_list:
        if tup[1] == 1:
            values_list.append(tup[0])
    
    # Return the list of values
    return values_list

def extract_name_and_file(path):
    filename = os.path.basename(path)
    name, ext = os.path.splitext(filename)
    species_name = path.split('/')[-2]
    return species_name, name

def split_on_capitals(string):
    # Split the string at every capital letter
    split_string = re.findall('[A-Z][^A-Z]*', string)
    # Join the split string with spaces
    new_string = ' '.join(split_string)
    
    return new_string

def clean_text(text):
    # Remove slashes and spaces around them
    text = re.sub(r'\s*/\s*', '', text)
    # Remove any remaining spaces
    text = re.sub(r'\s+', '', text)
    return text



### Open Data

In [54]:
folder_traits = "../../../data/OpenAI/Traits/"

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

In [55]:
# Testing
# test_dict_cleaned = {}
# test_dict_original = {}

# Define the path where the prompt results are stored
folder_prompts = "../../../data/OpenAI/PromptsResults/OriginalData/"

# Get a list of all folders in the path
species_folders = glob.glob(F"{folder_prompts}*")

# Create a defaultdict to store the prompt results for each species
prompt_results_dict = collections.defaultdict(dict)

# Loop through each species folder
for idx, species_folder in enumerate(species_folders[0:2]):
    # Extract the species name from the folder path

    # Split the path in head and tail pair 
    species_name = os.path.split(species_folder)[1].replace('_', ' ')
    # Append to dict
    prompt_results_dict[species_name] = {}

    # Testing
    # test_dict_cleaned[species_name] = {}
    # test_dict_original[species_name] = {}

    # Get a list of all JSON files in the folder
    json_list = glob.glob(F"{species_folder}/*")

    # Loop through each JSON file
    for json_file in (pbar := tqdm(json_list, leave=False, position=0)):
        # Update the progress bar description
        pbar.set_description(f"{idx}: {species_name}")

        # Create an empty dictionary to store the trait results
        trait_dict = {}

        # Extract the trait name and file name from the JSON file path
        _, trait = extract_name_and_file(json_file)
        # Clean up the trait name by splitting on capital letters
        trait = split_on_capitals(trait)

        # Read in the JSON file
        with open(json_file, 'r') as f:
            prompt_result = json.load(f)

        # Extract the list
        res = prompt_result['choices'][0]['message']['content']
        res = regex_cleaner(res)
        
        # print(res)
        # res = extract_list_from_string(res)

        # # print(res)
        # # print('---\n----')
        # # print(prompt_result['choices'][0]['message']['content'])

        # # Extract the correct values
        # res = extract_values_from_list(res)

        # Append to dict
        prompt_results_dict[species_name][trait.capitalize()] = res


In [56]:
prompt_results_dict

defaultdict(dict,
            {'Laguncularia racemosa': {'Petals corolla shape': '[(absent, 0), (free, 1), (tepals, 0), (tubular, 1)]',
              'Petals corolla colour': "[(green-white, yellow-white), 1), ('whitish', 1), ('white', 1), ('yellow', 0), ('yellow-green', 0), ('yellow, orange', 0), ('yellow, yellow-green', 0), ('yellow-green, green', 0), ('blue', 0), ('green, red', 0), ('greenish white', 0), ('red', 0), ('white, pink', 0), ('white, pink, purple', 0), ('white, whitish', 0), ('white, yellow-white', 0)]",
              'Sepals calyx shape': '[(connate, 0), (free, 1)]',
              'Leaf position': '[(opposite, 1)]',
              'Leaf margin': '[(crenate, 0), (entire, 1), (entire, crenate, 0), (entire, serrate, 0), (serrate, 0)]',
              'Inflorescence type': '[(panicle, 1), (spike, 1)]',
              'Leaf glands': '[(absent, 0), (gland on the end of petiole, 1), (glands near basis and central vein, 1), (glands near basis and leaf margin, 0), (translucent oil c

### Dataframe with GT

In [57]:
root = "../../../data/OpenAI/DataFrames/"

file = "DF_Andrei.csv"
df_Andrei = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Andrei = df_Andrei.rename(columns={'Thorns/spines': 'Thorns spines', 
                                      'Sepals / calyx shape': 'Sepals calyx shape', 
                                      'Petals / corolla shape': 'Petals corolla shape',
                                      'Petals / corolla number': 'Petals corolla number',
                                      'Petals / corolla colour': 'Petals corolla colour',
                                      'Sepals / calyx numer': 'Sepals calyx numer',})
df_Andrei_species = list(df_Andrei.index)


file = "DF_Daniel.csv"
df_Daniel = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')\
    .dropna()
df_Daniel_species = list(df_Daniel.index)

file = "DF_Pierre.csv"
df_Pierre = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Pierre_species = list(df_Pierre.index)

In [58]:
def extract_GT_traits(species, df):

    s = df.loc[species]
    GT_traits = list(s.where(s == 1).dropna().index)

    return GT_traits

def extract_ALL_traits(species, df):

    s = df.loc[species]
    traits = list(s.index)

    # Create a dict
    result = {}
    for key, value in traits:
        if key in result:
            result[key].append(value)
        else:
            result[key] = [value]

    return result


### Caribbean 

In [59]:
# Create an empty DataFrame with a multi-level index based on the columns of another DataFrame
df_Andrei_ChatGPT = pd.DataFrame(index=pd.MultiIndex.from_tuples(df_Andrei.columns)).T

# Loop through each species in a dictionary of ground truth traits
for species in prompt_results_dict.keys():

    # Extract the ground truth traits for the current species from a DataFrame
    ALL_traits = extract_ALL_traits(species, df_Andrei)

    # Loop through each trait and its corresponding value in the ground truth traits
    for trait, value in ALL_traits.items():

        # print(trait, 'Correct')
        ChatGPT_result = prompt_results_dict[species][trait]

        # If NaN
        if ChatGPT_result == 'NA':
            df_Andrei_ChatGPT.loc[species, (trait)] = np.NaN
            continue

        # Find the correct values by finding the intersection between the ground truth values and the ChatGPT result
        correct_values = list(set(value) & set(ChatGPT_result))
        incorrect_values = list(set(value) - set(correct_values))
        # Set a value of 1 in the DataFrame for each correct value for the current species and trait
        for correct_value in correct_values:
            df_Andrei_ChatGPT.loc[species, (trait, correct_value)] = 1
        for incorrect_value in incorrect_values:
            df_Andrei_ChatGPT.loc[species, (trait, incorrect_value)] = 0

folder_prompt_results = "../../../data/OpenAI/PromptsAnalysesData/"

# df_Andrei_ChatGPT.to_csv(F"{folder_prompt_results}caribbean_df_orig_ChatGPT.csv")
# df_Andrei.loc[df_Andrei_ChatGPT.index].to_csv(F"{folder_prompt_results}caribbean_df_orig_GT.csv")

In [60]:
df_Andrei.loc[df_Andrei_ChatGPT.index]

Life form      Leaf position                      \
                          liana tree     alternate alternate, opposite   
Laguncularia racemosa         0    1             0                   0   
Avicennia germinans           0    1             0                   0   

                                                      \
                      opposite opposite, whorls of 3   
Laguncularia racemosa        1                     0   
Avicennia germinans          1                     0   

                                                       Leaf composition  \
                      opposite, whorls of 3, alternate        3 palmate   
Laguncularia racemosa                                0                0   
Avicennia germinans                                  0                0   

                                               ... Fruit colour  \
                      3-5 palmate 3-5 pinnate  ...       orange   
Laguncularia racemosa           0           0  ...            0   
Avicennia germinans             0           0  ...            0   

                                                                        \
                      orange, yellow, red purple, black red red, black   
Laguncularia racemosa                   0             0   0          0   
Avicennia germinans                     0             0   0          0   

                                                                      \
                      red, purple-brown reddish purple whitish green   
Laguncularia racemosa                 0              0             1   
Avicennia germinans                   0              0             0   

                                                
                      yellow, orange yellowish  
Laguncularia racemosa              0         0  
Avicennia germinans                0         0  

[2 rows x 189 columns]

In [61]:
df_Andrei_ChatGPT

Life form      Leaf position                      \
                          liana tree     alternate alternate, opposite   
Laguncularia racemosa       0.0  0.0           0.0                 0.0   
Avicennia germinans         0.0  0.0           0.0                 0.0   

                                                      \
                      opposite opposite, whorls of 3   
Laguncularia racemosa      0.0                   0.0   
Avicennia germinans        0.0                   0.0   

                                                       Leaf composition  \
                      opposite, whorls of 3, alternate        3 palmate   
Laguncularia racemosa                              0.0              NaN   
Avicennia germinans                                0.0              NaN   

                                               ... Fruit colour  \
                      3-5 palmate 3-5 pinnate  ...       orange   
Laguncularia racemosa         NaN         NaN  ...          0.0   
Avicennia germinans           NaN         NaN  ...          0.0   

                                                                         \
                      orange, yellow, red purple, black  red red, black   
Laguncularia racemosa                 0.0           0.0  0.0        0.0   
Avicennia germinans                   0.0           0.0  0.0        0.0   

                                                                      \
                      red, purple-brown reddish purple whitish green   
Laguncularia racemosa               0.0            0.0           0.0   
Avicennia germinans                 0.0            0.0           0.0   

                                                
                      yellow, orange yellowish  
Laguncularia racemosa            0.0       0.0  
Avicennia germinans              0.0       0.0  

[2 rows x 189 columns]